In [1]:
import pandas as pd
import networkx as nx
import numpy as np
import glob

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# for notebook
%matplotlib inline

## Datasets

brasileirao_serie_a : 21 temporadas

In [2]:
# https://basedosdados.org/dataset/c861330e-bca2-474d-9073-bc70744a1b23?table=18835b0d-233e-4857-b454-1fa34a81b4fa

brasileirao = pd.read_csv('../data/brasileirao_a/brasileirao_serie_a.csv', sep=',')
df = brasileirao.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8079 entries, 0 to 8078
Data columns (total 35 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ano_campeonato                  8079 non-null   int64  
 1   data                            8079 non-null   object 
 2   rodada                          8079 non-null   int64  
 3   estadio                         8068 non-null   object 
 4   arbitro                         6369 non-null   object 
 5   publico                         6368 non-null   float64
 6   publico_max                     3817 non-null   float64
 7   time_mandante                   8079 non-null   object 
 8   time_visitante                  8079 non-null   object 
 9   tecnico_mandante                5926 non-null   object 
 10  tecnico_visitante               5926 non-null   object 
 11  colocacao_mandante              6369 non-null   float64
 12  colocacao_visitante             63

In [3]:
df.head(20)

,ano_campeonato,data,rodada,estadio,arbitro,publico,publico_max,time_mandante,time_visitante,tecnico_mandante,...,chutes_bola_parada_mandante,chutes_bola_parada_visitante,defesas_mandante,defesas_visitante,impedimentos_mandante,impedimentos_visitante,chutes_mandante,chutes_visitante,chutes_fora_mandante,chutes_fora_visitante
0,2017,2017-05-21,2,Arena Condá,André Luiz de Freitas Castro,10323.0,22600.0,Chapecoense,Palmeiras,Vágner Mancini,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017,2017-05-30,3,Arena Condá,Marcelo de Lima Henrique,6188.0,22600.0,Chapecoense,Avaí FC,Vágner Mancini,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017,2017-06-09,5,Arena Condá,Rodolpho Toski Marques,14403.0,22600.0,Chapecoense,Grêmio,Vágner Mancini,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017,2017-06-15,7,Arena Condá,Jailson Macedo Freitas,10033.0,22600.0,Chapecoense,Vasco da Gama,Vágner Mancini,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017,2017-06-18,8,Arena Condá,Marcelo Aparecido Ribeiro de Souza,11603.0,22600.0,Chapecoense,Botafogo,Vágner Mancini,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2017,2017-06-26,10,Arena Condá,João Batista de Arruda,8502.0,22600.0,Chapecoense,Atlético-MG,Vágner Mancini,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2017,2017-07-09,12,Arena Condá,Caio Max Augusto Vieira,9689.0,22600.0,Chapecoense,Atlético-PR,Vinícius Eutrópio,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2017,2017-07-16,14,Arena Condá,Wagner do Nascimento Magalhães,10742.0,22600.0,Chapecoense,São Paulo,Vinícius Eutrópio,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2017,2017-07-30,17,Arena Condá,Marcelo Aparecido Ribeiro de Souza,9744.0,22600.0,Chapecoense,Atlético-GO,Vinícius Eutrópio,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2017,2017-08-03,18,Arena Condá,Flavio Rodrigues de Souza,3466.0,25000.0,Chapecoense,EC Bahia,Vinícius Eutrópio,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.tail(20)

,ano_campeonato,data,rodada,estadio,arbitro,publico,publico_max,time_mandante,time_visitante,tecnico_mandante,...,chutes_bola_parada_mandante,chutes_bola_parada_visitante,defesas_mandante,defesas_visitante,impedimentos_mandante,impedimentos_visitante,chutes_mandante,chutes_visitante,chutes_fora_mandante,chutes_fora_visitante
8059,2011,2011-08-31,20,Estádio Governador Magalhães Pinto,Jailson Macedo Freitas,14738.0,NaN,Cruzeiro,Figueirense FC,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8060,2011,2011-09-24,26,Estádio Urbano Caldeira,Héber Lopes,7059.0,NaN,Santos FC,Figueirense FC,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8061,2011,2011-10-12,29,Estádio Olímpico Monumental,Rodrigo Braghetto,16449.0,NaN,Grêmio,Figueirense FC,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8062,2011,2011-10-12,29,Estádio Municipal Paulo Machado de Carvalho,Elmo Alves Resende Cunha,32450.0,NaN,Corinthians,Botafogo,Tite,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8063,2011,2011-07-17,10,Estádio Beira-Rio,Wagner Reway,11800.0,NaN,Internacional,São Paulo,Falcão,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8064,2011,2011-10-22,31,Estádio Jornalista Mário Filho,Jean Pierre Goncalves Lima,24143.0,NaN,Fluminense,Atlético-MG,Abel Braga,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8065,2011,2011-11-16,35,Estádio Jornalista Mário Filho,Franscisco Carlos do Nascimento,11395.0,NaN,Fluminense,Grêmio,Abel Braga,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8066,2011,2011-10-30,32,Estádio Olímpico Monumental,Evandro Rogerio Roman,44781.0,NaN,Grêmio,Flamengo,Celso Roth,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8067,2011,2011-08-13,16,Estádio Olímpico Nilton Santos,André Luiz de Freitas Castro,13101.0,NaN,Botafogo,América-MG,Caio Júnior,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8068,2011,2011-09-08,22,Estádio Governador Roberto Santos,Felipe Gomes da Silva,17977.0,NaN,EC Bahia,Grêmio,Joel Santana,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.columns

Index(['ano_campeonato', 'data', 'rodada', 'estadio', 'arbitro', 'publico',
       'publico_max', 'time_mandante', 'time_visitante', 'tecnico_mandante',
       'tecnico_visitante', 'colocacao_mandante', 'colocacao_visitante',
       'valor_equipe_titular_mandante', 'valor_equipe_titular_visitante',
       'idade_media_titular_mandante', 'idade_media_titular_visitante',
       'gols_mandante', 'gols_visitante', 'gols_1_tempo_mandante',
       'gols_1_tempo_visitante', 'escanteios_mandante', 'escanteios_visitante',
       'faltas_mandante', 'faltas_visitante', 'chutes_bola_parada_mandante',
       'chutes_bola_parada_visitante', 'defesas_mandante', 'defesas_visitante',
       'impedimentos_mandante', 'impedimentos_visitante', 'chutes_mandante',
       'chutes_visitante', 'chutes_fora_mandante', 'chutes_fora_visitante'],
      dtype='object')

In [6]:
df = df[[
    'ano_campeonato',
    'time_mandante', 
    'time_visitante', 
    'gols_mandante', 
    'gols_visitante'
]]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8079 entries, 0 to 8078
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ano_campeonato  8079 non-null   int64  
 1   time_mandante   8079 non-null   object 
 2   time_visitante  8079 non-null   object 
 3   gols_mandante   8078 non-null   float64
 4   gols_visitante  8078 non-null   float64
dtypes: float64(2), int64(1), object(2)
memory usage: 315.7+ KB


In [7]:
df.rename(columns = {
    'ano_campeonato':'year', 
    'time_mandante':'home', 
    'time_visitante':'away', 
    'gols_mandante':'home_goals', 
    'gols_visitante':'away_goals'
    }, 
    inplace = True) 

In [8]:
df = df[['year','home','away','home_goals','away_goals']]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8079 entries, 0 to 8078
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   year        8079 non-null   int64  
 1   home        8079 non-null   object 
 2   away        8079 non-null   object 
 3   home_goals  8078 non-null   float64
 4   away_goals  8078 non-null   float64
dtypes: float64(2), int64(1), object(2)
memory usage: 315.7+ KB


In [9]:
df = df.dropna(how='any',axis=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8078 entries, 0 to 8078
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   year        8078 non-null   int64  
 1   home        8078 non-null   object 
 2   away        8078 non-null   object 
 3   home_goals  8078 non-null   float64
 4   away_goals  8078 non-null   float64
dtypes: float64(2), int64(1), object(2)
memory usage: 378.7+ KB


In [10]:
print(sorted(df['year'].unique()))
print("#temporadas = ", len(df['year'].unique()))

[2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
#temporadas =  21


In [11]:
#temp = ['2017-18', '2018-19', '2019-20', '2020-21', '2021-22', '2022-23']
#df = df.loc[df['year'].isin(temp)].reset_index()

In [12]:
vhome = list(df['home'].unique())
print(len(vhome))

vaway = list(df['away'].unique())
print(len(vaway))

vertice = set(vhome) | set(vaway)
print(len(vertice))
print(list(vertice))

46
46
46
['Atlético-GO', 'Botafogo', 'Grêmio', 'Santa Cruz', 'Ceará SC', 'São Caetano', 'Goiás', 'Guarani', 'EC Bahia', 'Santo André', 'Barueri', 'Sport Recife', 'Figueirense FC', 'Náutico', 'Brasiliense-DF', 'EC Vitória', 'Santos FC', 'Joinville-SC', 'Coritiba FC', 'Paraná', 'Juventude', 'Internacional', 'Criciúma EC', 'Avaí FC', 'São Paulo', 'Athletico-PR', 'Fortaleza', 'América-RN', 'Portuguesa', 'Fluminense', 'Paysandu SC', 'Vasco da Gama', 'CSA', 'Cuiabá-MT', 'Cruzeiro', 'Corinthians', 'Atlético-MG', 'Ipatinga FC', 'Palmeiras', 'Atlético-PR', 'Ponte Preta', 'América-MG', 'Santos', 'Chapecoense', 'RB Bragantino', 'Flamengo']


In [13]:
df['home_win'] = 0.0
df['away_win'] = 0.0
df['draw'] = 0.0

df.loc[df['home_goals'] > df['away_goals'], 'home_win'] = 1.0
df.loc[df['home_goals'] < df['away_goals'], 'away_win'] = 1.0
df.loc[df['home_goals'] == df['away_goals'], 'draw'] = 1.0

df.head()

,year,home,away,home_goals,away_goals,home_win,away_win,draw
0,2017,Chapecoense,Palmeiras,1.0,0.0,1.0,0.0,0.0
1,2017,Chapecoense,Avaí FC,2.0,0.0,1.0,0.0,0.0
2,2017,Chapecoense,Grêmio,3.0,6.0,0.0,1.0,0.0
3,2017,Chapecoense,Vasco da Gama,2.0,1.0,1.0,0.0,0.0
4,2017,Chapecoense,Botafogo,0.0,2.0,0.0,1.0,0.0


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8078 entries, 0 to 8078
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   year        8078 non-null   int64  
 1   home        8078 non-null   object 
 2   away        8078 non-null   object 
 3   home_goals  8078 non-null   float64
 4   away_goals  8078 non-null   float64
 5   home_win    8078 non-null   float64
 6   away_win    8078 non-null   float64
 7   draw        8078 non-null   float64
dtypes: float64(5), int64(1), object(2)
memory usage: 568.0+ KB


In [15]:
df2 = df.groupby(
    ['home', 'away'],
    as_index=False,
    sort=False).agg({'home_goals':'sum', 'away_goals':'sum', 'home_win':'sum', 'away_win':'sum', 'draw' : 'sum'})

df2.head(20)

,home,away,home_goals,away_goals,home_win,away_win,draw
0,Chapecoense,Palmeiras,11.0,8.0,3.0,3.0,1.0
1,Chapecoense,Avaí FC,3.0,0.0,2.0,0.0,1.0
2,Chapecoense,Grêmio,10.0,16.0,1.0,4.0,2.0
3,Chapecoense,Vasco da Gama,5.0,4.0,2.0,1.0,1.0
4,Chapecoense,Botafogo,4.0,5.0,2.0,3.0,0.0
5,Chapecoense,Atlético-MG,7.0,7.0,2.0,2.0,2.0
6,Chapecoense,Atlético-PR,6.0,2.0,2.0,0.0,3.0
7,Chapecoense,São Paulo,6.0,5.0,3.0,2.0,2.0
8,Chapecoense,Atlético-GO,1.0,3.0,0.0,2.0,0.0
9,Chapecoense,EC Bahia,4.0,5.0,1.0,1.0,3.0


In [16]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1553 entries, 0 to 1552
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   home        1553 non-null   object 
 1   away        1553 non-null   object 
 2   home_goals  1553 non-null   float64
 3   away_goals  1553 non-null   float64
 4   home_win    1553 non-null   float64
 5   away_win    1553 non-null   float64
 6   draw        1553 non-null   float64
dtypes: float64(5), object(2)
memory usage: 85.1+ KB


In [17]:
lista = []
for keys, infos in df2.iterrows():
    time1 = infos.home
    time2 = infos.away
    t = df2.loc[(df2['home'] == time2) & (df2['away'] == time1)][['home','away']]
    
    if len(t) == 0:
        lista.append(time1)
        print(time2,time1)

print("#jogos a serem adicionados:", len(lista))

Internacional Brasiliense-DF
Brasiliense-DF Santos FC
Brasiliense-DF Figueirense FC
Ponte Preta Brasiliense-DF
Criciúma EC Paraná
Guarani EC Bahia
São Caetano EC Bahia
Criciúma EC Fortaleza
Cuiabá-MT Vasco da Gama
#jogos a serem adicionados: 9


In [18]:
time1 = "Internacional"
time2 = "Brasiliense-DF"

t = df2.loc[(df2['home'] == time1) & (df2['away'] == time2)][['home','away','home_goals','away_goals']]
print(t.values)

t = df2.loc[(df2['home'] == time2) & (df2['away'] == time1)][['home','away','home_goals','away_goals']]
print(t.values)

[]
[['Brasiliense-DF' 'Internacional' 0.0 0.0]]


In [19]:
df2.head(20)

,home,away,home_goals,away_goals,home_win,away_win,draw
0,Chapecoense,Palmeiras,11.0,8.0,3.0,3.0,1.0
1,Chapecoense,Avaí FC,3.0,0.0,2.0,0.0,1.0
2,Chapecoense,Grêmio,10.0,16.0,1.0,4.0,2.0
3,Chapecoense,Vasco da Gama,5.0,4.0,2.0,1.0,1.0
4,Chapecoense,Botafogo,4.0,5.0,2.0,3.0,0.0
5,Chapecoense,Atlético-MG,7.0,7.0,2.0,2.0,2.0
6,Chapecoense,Atlético-PR,6.0,2.0,2.0,0.0,3.0
7,Chapecoense,São Paulo,6.0,5.0,3.0,2.0,2.0
8,Chapecoense,Atlético-GO,1.0,3.0,0.0,2.0,0.0
9,Chapecoense,EC Bahia,4.0,5.0,1.0,1.0,3.0


In [20]:
df2.tail(20)

,home,away,home_goals,away_goals,home_win,away_win,draw
1533,Avaí FC,Juventude,1.0,2.0,0.0,1.0,0.0
1534,Botafogo,Cuiabá-MT,0.0,2.0,0.0,1.0,0.0
1535,América-MG,Avaí FC,5.0,3.0,1.0,0.0,1.0
1536,Cuiabá-MT,Goiás,2.0,3.0,0.0,1.0,1.0
1537,Palmeiras,Criciúma EC,1.0,0.0,1.0,0.0,0.0
1538,Criciúma EC,Sport Recife,2.0,2.0,0.0,0.0,1.0
1539,Sport Recife,Criciúma EC,2.0,0.0,1.0,0.0,0.0
1540,Chapecoense,Criciúma EC,1.0,1.0,0.0,0.0,1.0
1541,Criciúma EC,Chapecoense,1.0,0.0,1.0,0.0,0.0
1542,Vasco da Gama,Cuiabá-MT,1.0,0.0,1.0,0.0,0.0


In [21]:
df2["weight1"] = 0.0
df2["weight2"] = 0.0

In [22]:
for keys, infos in df2.iterrows():
    time1 = infos.home
    time2 = infos.away

    t1 =  df2.loc[(df2['home'] == time1) & (df2['away'] == time2)][['away_goals','away_win','draw']].values
    t2 =  df2.loc[(df2['away'] == time1) & (df2['home'] == time2)][['home_goals','home_win','draw']].values

    g1 = p1 = 0
    for i1 in t1:
        g1 = i1[0]
        p1 = 3*i1[1] + 1*i1[2]

    g2 = p2 = 0
    for i2 in t2:
        g2 = i2[0]
        p2 = 3*i2[1] + 1*i2[2]

    df2.loc[keys,'weight1'] = g1 + g2
    df2.loc[keys,'weight2'] = p1 + p2

In [23]:
df2.head(20)

,home,away,home_goals,away_goals,home_win,away_win,draw,weight1,weight2
0,Chapecoense,Palmeiras,11.0,8.0,3.0,3.0,1.0,19.0,26.0
1,Chapecoense,Avaí FC,3.0,0.0,2.0,0.0,1.0,3.0,7.0
2,Chapecoense,Grêmio,10.0,16.0,1.0,4.0,2.0,27.0,27.0
3,Chapecoense,Vasco da Gama,5.0,4.0,2.0,1.0,1.0,10.0,10.0
4,Chapecoense,Botafogo,4.0,5.0,2.0,3.0,0.0,9.0,19.0
5,Chapecoense,Atlético-MG,7.0,7.0,2.0,2.0,2.0,20.0,19.0
6,Chapecoense,Atlético-PR,6.0,2.0,2.0,0.0,3.0,12.0,14.0
7,Chapecoense,São Paulo,6.0,5.0,3.0,2.0,2.0,16.0,18.0
8,Chapecoense,Atlético-GO,1.0,3.0,0.0,2.0,0.0,5.0,8.0
9,Chapecoense,EC Bahia,4.0,5.0,1.0,1.0,3.0,10.0,13.0


In [24]:
df2.tail(20)

,home,away,home_goals,away_goals,home_win,away_win,draw,weight1,weight2
1533,Avaí FC,Juventude,1.0,2.0,0.0,1.0,0.0,3.0,4.0
1534,Botafogo,Cuiabá-MT,0.0,2.0,0.0,1.0,0.0,4.0,6.0
1535,América-MG,Avaí FC,5.0,3.0,1.0,0.0,1.0,6.0,5.0
1536,Cuiabá-MT,Goiás,2.0,3.0,0.0,1.0,1.0,4.0,7.0
1537,Palmeiras,Criciúma EC,1.0,0.0,1.0,0.0,0.0,2.0,0.0
1538,Criciúma EC,Sport Recife,2.0,2.0,0.0,0.0,1.0,4.0,4.0
1539,Sport Recife,Criciúma EC,2.0,0.0,1.0,0.0,0.0,2.0,1.0
1540,Chapecoense,Criciúma EC,1.0,1.0,0.0,0.0,1.0,2.0,4.0
1541,Criciúma EC,Chapecoense,1.0,0.0,1.0,0.0,0.0,1.0,1.0
1542,Vasco da Gama,Cuiabá-MT,1.0,0.0,1.0,0.0,0.0,0.0,0.0


In [25]:
for keys, infos in df2.iterrows():
    time1 = infos.home
    time2 = infos.away

    t2 = df2.loc[(df2['home'] == time2) & (df2['away'] == time1)]
    
    if len(t2) == 0:
        t1 =  df2.loc[(df2['home'] == time1) & (df2['away'] == time2)][['home_goals','home_win','draw']].values

        w1 = w2 = 0
        for i1 in t1:
            w1 = i1[0]
            w2 = 3*i1[1] + 1*i1[2]

        new = [time2, time1,np.nan,np.nan,0,0,0,w1,w2]

        new = pd.DataFrame(columns=df2.columns, data=[new])
        
        df2 = pd.concat([df2, new], axis=0)

In [26]:
df2.head(20)

,home,away,home_goals,away_goals,home_win,away_win,draw,weight1,weight2
0,Chapecoense,Palmeiras,11.0,8.0,3.0,3.0,1.0,19.0,26.0
1,Chapecoense,Avaí FC,3.0,0.0,2.0,0.0,1.0,3.0,7.0
2,Chapecoense,Grêmio,10.0,16.0,1.0,4.0,2.0,27.0,27.0
3,Chapecoense,Vasco da Gama,5.0,4.0,2.0,1.0,1.0,10.0,10.0
4,Chapecoense,Botafogo,4.0,5.0,2.0,3.0,0.0,9.0,19.0
5,Chapecoense,Atlético-MG,7.0,7.0,2.0,2.0,2.0,20.0,19.0
6,Chapecoense,Atlético-PR,6.0,2.0,2.0,0.0,3.0,12.0,14.0
7,Chapecoense,São Paulo,6.0,5.0,3.0,2.0,2.0,16.0,18.0
8,Chapecoense,Atlético-GO,1.0,3.0,0.0,2.0,0.0,5.0,8.0
9,Chapecoense,EC Bahia,4.0,5.0,1.0,1.0,3.0,10.0,13.0


In [27]:
df2.tail(20)

,home,away,home_goals,away_goals,home_win,away_win,draw,weight1,weight2
1542,Vasco da Gama,Cuiabá-MT,1.0,0.0,1.0,0.0,0.0,0.0,0.0
1543,Cuiabá-MT,Cruzeiro,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1544,Cruzeiro,RB Bragantino,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1545,Vasco da Gama,Santos,0.0,1.0,0.0,1.0,0.0,5.0,6.0
1546,Santos,Vasco da Gama,4.0,1.0,1.0,0.0,0.0,1.0,0.0
1547,Cruzeiro,Santos,2.0,1.0,1.0,0.0,0.0,1.0,0.0
1548,Cruzeiro,Cuiabá-MT,0.0,1.0,0.0,1.0,0.0,1.0,4.0
1549,Santos,Cruzeiro,0.0,3.0,0.0,1.0,0.0,5.0,6.0
1550,RB Bragantino,Cruzeiro,0.0,3.0,0.0,1.0,0.0,3.0,4.0
1551,Figueirense FC,Ceará SC,1.0,1.0,0.0,0.0,1.0,2.0,2.0


In [28]:
time1 = "Internacional"
time2 = "Brasiliense-DF"

t = df2.loc[(df2['home'] == time1) & (df2['away'] == time2)][['home','away','home_goals','away_goals','home_win', 'away_win', 'draw','weight1','weight2']]
print(t.values)

t = df2.loc[(df2['home'] == time2) & (df2['away'] == time1)][['home','away','home_goals','away_goals','home_win', 'away_win', 'draw','weight1','weight2']]
print(t.values)

[['Internacional' 'Brasiliense-DF' nan nan 0.0 0.0 0.0 0.0 1.0]]
[['Brasiliense-DF' 'Internacional' 0.0 0.0 0.0 0.0 1.0 0.0 1.0]]


In [29]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1562 entries, 0 to 0
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   home        1562 non-null   object 
 1   away        1562 non-null   object 
 2   home_goals  1553 non-null   float64
 3   away_goals  1553 non-null   float64
 4   home_win    1562 non-null   float64
 5   away_win    1562 non-null   float64
 6   draw        1562 non-null   float64
 7   weight1     1562 non-null   float64
 8   weight2     1562 non-null   float64
dtypes: float64(7), object(2)
memory usage: 122.0+ KB


In [30]:
df3 = df2[['home','away','weight1','weight2']]

df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1562 entries, 0 to 0
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   home     1562 non-null   object 
 1   away     1562 non-null   object 
 2   weight1  1562 non-null   float64
 3   weight2  1562 non-null   float64
dtypes: float64(2), object(2)
memory usage: 61.0+ KB


In [31]:
df3.head(20)

,home,away,weight1,weight2
0,Chapecoense,Palmeiras,19.0,26.0
1,Chapecoense,Avaí FC,3.0,7.0
2,Chapecoense,Grêmio,27.0,27.0
3,Chapecoense,Vasco da Gama,10.0,10.0
4,Chapecoense,Botafogo,9.0,19.0
5,Chapecoense,Atlético-MG,20.0,19.0
6,Chapecoense,Atlético-PR,12.0,14.0
7,Chapecoense,São Paulo,16.0,18.0
8,Chapecoense,Atlético-GO,5.0,8.0
9,Chapecoense,EC Bahia,10.0,13.0


In [32]:
df3.tail(20)

,home,away,weight1,weight2
1542,Vasco da Gama,Cuiabá-MT,0.0,0.0
1543,Cuiabá-MT,Cruzeiro,0.0,1.0
1544,Cruzeiro,RB Bragantino,0.0,1.0
1545,Vasco da Gama,Santos,5.0,6.0
1546,Santos,Vasco da Gama,1.0,0.0
1547,Cruzeiro,Santos,1.0,0.0
1548,Cruzeiro,Cuiabá-MT,1.0,4.0
1549,Santos,Cruzeiro,5.0,6.0
1550,RB Bragantino,Cruzeiro,3.0,4.0
1551,Figueirense FC,Ceará SC,2.0,2.0


In [33]:
df3.to_csv('../data/work1/brasileirao_a_2003_2012.csv', index=False, sep=';')